# Méthode applicable de façon totalement automatique:

Nous voulons maintenant développer une fonction pouvant traiter de façon totalement automatique tout ce que l'on vient de faire. 

Les entrées de cette fonction sont:
- Les deux datasets
- Les noms respectifs des colonnes communes à ces deux datasets

Les sorties de cette fonction sont: 
- Le Dataset avec toutes les données associées trouvées
- Le Dataset contenant exclusivement les résultats trouvés lors de l'étape 2, qui sont donc moins 'sûrs', avec le score TF-IDF associé, ce qui permet de trier par ordre croissant de ce score et par conséquent de vérifier en premier les résultats les 'moins' certains. 

**Etapes nécessaires:**

    1) 



Affichages de cette fonction:
- Nombre d'associations trouvées à chaque étape

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import unidecode

#Necessary imports for tfidf and cosine similarity:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

## Fonction à mettre en place:

Dans la présente partie de ce notebook, nous allons développer des fonctions permettant de mettre en place une fonction globale mettant en place le rapprochement de deux datasets de façon automatique, tel qu'explicité en introduction de ce notebook.

La plupart des fonctions developpées dans cette partie l'ont déjà été dans [ce notebook de travail](https://github.com/adrihans/musees_de_france_app/blob/master/Linking_datasets.ipynb).

### Fonction de nettoyage

In [2]:
def format_string(string):
    """
    function 'format_string' to transform a given string 
    
    @Inputs : 
    -string, str
    
    @Outputs : 
    -string, str
    """
    string=unidecode.unidecode(string.strip().lower())
    string=string.replace('-',' ')
    string=string.replace('\'', " ")
    return re.sub(' +', ' ', string)

### Inclusivité entre les colonnes

In [3]:
def merge_inclusivity(df_join, df_right, col_left, col_right, other_col_left, other_col_right):
    #Copy of the df_join:
    df_join_copy=df_join.copy()
    columns_to_update=df_right.columns
    #Loop over the nans elements: 
    for index_na, element_na in df_join[df_join[columns_to_update[1]].isna()].iterrows():
        for index_right, element_right in df_right.iterrows():
            #If condition
            if (((element_na[col_left] in element_right[col_right])
            or (element_right[col_right] in element_na[col_left]))
                
            and ((element_na[other_col_left] in element_right[other_col_right])
            or (element_right[other_col_right] in element_na[other_col_left]))):
                
                #Updating columns:
                for col_up in columns_to_update:
                    df_join_copy[col_up].iloc[index_na]=element_right[col_up]
                    
    return df_join_copy

### Méthodes avec TF-IDF

#### Première méthode: Une seule colonne dans la comparaison au sens de TF-IDF et cosine similarity

In [4]:
def find_return_answers(df_missing_left, df_right, column_left,
                        column_right, other_column_left, other_column_right, top_n=5):
    """
    This function, from one dataset with nans, and the other to be associated with the first one, 
    returns a new dataFrame with what has been found
    
    @Inputs:
    -df_missing_left
    -df_right
    -column_left
    -column_right
    -Other_column_left
    -Other_column_right
    -top_n, int, optional,the number of top_n answers to be retreived
    @Outputs:
    -df_new_values
    """
    #Initializing a tfidf model:
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=0, norm='l2', encoding='latin-1', ngram_range=(1,1))
    #Applying the new tfidf model on the right Dataframe and on the column specified:
    tfidf_matrix_right = tfidf.fit_transform(df_right[column_right]).toarray()
    #Creating the DataFrame this function is going to be returning:
    #Firstly, this DataFrame countains only the uncomplete rows:
    df_new_values=df_missing_left.copy()
    #We add the sim_score column to save the similarity scores:
    df_new_values['sim_score']=0
    #Loop over the indexes and elements in the new DataFrame:
    
    
    
    #Modifying this line:
    for index,element in df_new_values[df_new_values[column_right].isna()].iterrows():
        #Defining the query for each row:
        query=element[column_left]
        #Transforming the query from the tfidf_model:
        tfidf_query=tfidf.transform([query]).toarray()
        #Calculating the cosine similarities between the query and the tfidf_matrix_right
        cosine_similarities = cosine_similarity(tfidf_query,tfidf_matrix_right).flatten()
        #We sort the documents from the similarity to the query :
        related_docs_indices = [i for i in cosine_similarities.argsort()[::-1]]
        #We retrieve the top n documents most similar to the given query :
        indexes_scores = [(index_t, cosine_similarities[index]) for index_t in related_docs_indices][0:top_n]
        #Loop over the index and scores of the answers:
        for index_ans, score_ans in indexes_scores:
            #Verifying over the other column:
            if ((element[other_column_left] in df_right.iloc[index_ans][other_column_right])
                or 
                (df_right.iloc[index_ans][other_column_right] in element[other_column_left])):
                
        
                
                #This answer is then satisfying our conditions, so we can merge the two rows.                
                df_new_values.loc[index]=df_new_values.loc[index].fillna(df_right.iloc[index_ans])
                #We add, for information, the score:
                df_new_values.loc[index,'sim_score']=score_ans
                #We break the loop since we found an answer:
                break
    return df_new_values

#### Seconde méthode: Plusieurs colonnes dans la comparaison au sens de TF-IDF et de cosine similarity

In [5]:
def find_return_answers_deux(df_missing_left, df_right, column_left,
                        column_right, other_column_left, other_column_right, top_n=5):
    """
    This function, from one dataset with nans, and the other to be associated with the first one, 
    returns a new dataFrame with what has been found
    
    @Inputs:
    -df_missing_left
    -df_right
    -column_left
    -column_right
    -Other_column_left
    -Other_column_right
    -top_n, int, optional,the number of top_n answers to be retreived
    @Outputs:
    -df_new_values
    """
    #Initializing a tfidf model:
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=0, norm='l2', encoding='latin-1', ngram_range=(1,1))
    #Applying the new tfidf model on the right Dataframe and on the column specified:
    tfidf_matrix_right = tfidf.fit_transform(df_right[column_right].str.cat(df_right[other_column_right].values.astype(str), sep=' ').astype(str)).toarray()
    #Creating the DataFrame this function is going to be returning:
    #Firstly, this DataFrame countains only the uncomplete rows:
    df_new_values=df_missing_left.copy()
    #We add a column representing the similarity score:
    df_new_values['sim_score']=0
    
    
    
    
    
    #Modifying this line:
    #Loop over the indexes and elements in the new DataFrame:
    for index,element in df_new_values[df_new_values[column_right].isna()].iterrows():
        #Defining the query for each row:
        query=element[column_left]+' '+element[other_column_left]
        #Transforming the query from the tfidf_model:
        tfidf_query=tfidf.transform([query]).toarray()
        #Calculating the cosine similarities between the query and the tfidf_matrix_right
        cosine_similarities = cosine_similarity(tfidf_query,tfidf_matrix_right).flatten()
        #We sort the documents from the similarity to the query :
        related_docs_indices = [i for i in cosine_similarities.argsort()[::-1]]
        #We retrieve the top n documents most similar to the given query :
        indexes_scores = [(index_t, cosine_similarities[index]) for index_t in related_docs_indices][0:top_n]
        #Loop over the index and scores of the answers:
        for index_ans, score_ans in indexes_scores:
            #Verifying over the other column:
            if ((element[other_column_left] in df_right.iloc[index_ans][other_column_right]) 
                or 
                (df_right.iloc[index_ans][other_column_right] in element[other_column_left])):
                
                #This answer is then satisfying our conditions, so we can merge the two rows.                
                df_new_values.loc[index]=df_new_values.loc[index].fillna(df_right.iloc[index_ans])
                #We add, for information, the score:
                df_new_values.loc[index,'sim_score']=score_ans
                #We break the loop since we found an answer:
                break
                
    return df_new_values

### Fonction d'affichage:

Cette fonction, `print_step_results`, permet l'affichage des résultats après chaque étape réalisée dans la fonction. 

Cela permet de ne pas surcharger la fonction finale, puisqu'un code similaire sera exécuté à chaque fois.

In [6]:
def print_steps_results(df,one_col_right,step_name):
    print("------------------", step_name, "-------------------")
    number_rows=df.shape[0]
    number_to_find=df[one_col_right].isna().sum()
    number_found=number_rows-number_to_find
    print("Au début de cette étape, il y avait {n_rows} éléments à trouver.\n\
    Appliquer cette étape a permis de trouver {n_found} éléments supplémentaires.\n"\
          .format(n_rows=number_rows,n_found=number_found))
    print('Il reste maintenant {n_to_find} éléments à trouver.'.format(n_to_find=number_to_find))
    return None

### Fonction d'interactions avec l'utilisateur:

Cette fonction, `func_interact`, permet d'intéragir avec l'utilisateur, pour lui permettre de modifier les résultats si certains ne lui semblent pas être les bons. 

Il peut alors, en spécifiant le nom de la colonne pour laquelle il veut préciser les éléments à supprimer, et en entrant ces éléments, de les supprimer dans le DataFrame.

In [7]:
def func_interact(df, columns_right):
    #Initializing the answers:
    ans_mistake=" "
    col=" "
    #Copying the Dataframe in input:
    df_copy=df.copy()
    
    while ans_mistake != "o" or ans_mistake != "n":
        #Asking the user if he notices any mistakes:
        print('Voyez vous des erreurs ? O ou N ?')
        ans_mistake=input().strip().lower()
        if ans_mistake=='o':
            while col not in df_copy.columns:
                print('Pour quelle colonne voulez vous entrer les valeurs EXACTES?')
                col=input()
                if col not in df_copy.columns:
                    print("Le nom de colonne n'est pas exact. Merci de répondre à nouveau.")
            print("Quelles sont les valeurs que vous voulez supprimer ? ")
            List_element=input()
            #Transformation de la string de la liste en une liste:
            List_element=ast.literal_eval(List_element)
            #Loop over the elements inside the list:
            for el in List_element:
                
                
                
                
                
                
                #Replacing by nans the unsatisfying results:
                df_copy.loc[df_copy[col]==el, 'columns_right'] = np.nan
                
                
                
                
                
                
                
                
                
        elif ans_mistake=='n':
            print("Content qu'il n'y ait plus d'erreurs !")
        else:
            print("Votre réponse n'est ni 'O' ni 'N', merci de répondre à nouveau.")
            
        return df_copy

## Mise en place de la fonction globale

Plusieurs étapes sont ici à considérer:
1. **Nettoyer les champs des colonnes**


2. **Appliquer une première jointure, avec égalité parfaite après le nettoyage de la précédente étape**
    1. Afficher les résultats 
    
    
3. **Appliquer une jointure en testant l'inclusivité**
    1. Afficher les résultats
    
    
4. **Appliquer la première méthode avec TF-IDF**
    1. Afficher les résultats
    2. Afficher les éléments trouvés *en plus* par rapport à l'étape précéddente
    
    
5. **Appliquer la deuxième méthode avec TF-IDF**
    1. Afficher les résultats
    2. Afficher les éléments trouvés *en plus* par rapport à l'étape précédente
    
    
6. **Demander quels résultats ne semblent pas satisfaisants**


7. **Afficher tous les résultats**


8. **Demander si certains résultats semblent à supprimer à nouveau**

La fonction, nommé `merging_two_datasets`, est développée ci-après:

In [8]:
def merging_two_datasets(df_left, df_right, columns_left, columns_right, interact=False):
    """
    Function merging_two_datasets, taking the 
    @Inputs:
    -df_left
    
    -df_right
    
    
    -columns_left
    The id of the df_left dataframe MUST BE the first element of this list
    
    
    -columns_right
    
    -interact, boolean,
    @Outputs:
    -df_merged
    """
    #We take the name of the id column for both DataFrame
    col_id_left=columns_left[0]
    col_id_right=columns_right[0]
    
    #Verifying there is one Id per row and that the number of Ids is equal to the number of rows:
    #We also make a copy of the input Dataframe to avoid modifying them by mistake
    df_left_travail=df_left.groupby(col_id_left).first().reset_index()[columns_left].copy()
    df_right_travail=df_right.groupby(col_id_right).first().reset_index()[columns_right].copy()
    #Cleaning the datasets:
    for col_left in columns_left[1:]:
        df_left_travail[col_left]=df_left_travail[col_left].astype(str).apply(format_string)
    for col_right in columns_right[1:]:
        df_right_travail[col_right]=df_right_travail[col_right].astype(str).apply(format_string)    
    #Trying the first merge, just by cleaning the Dataframes:
    df_first_left_merge=pd.merge(left=df_left_travail,
                           right=df_right_travail,
                           left_on=columns_left[1:],
                           right_on=columns_right[1:],
                           how='left')
    #Printing the results:
    print_steps_results(df=df_first_left_merge,
                        one_col_right=columns_right[1],
                        step_name="Première étape : merge simple après nettoyage des données")
    
    
    #Trying the second merge, dealing with inclusivity:
    df_inclusivity_left_merge=merge_inclusivity(df_join=df_first_left_merge, 
                                                df_right=df_right_travail, 
                                                col_left=columns_left[1], 
                                                col_right=columns_right[1], 
                                                other_col_left=columns_left[2], 
                                                other_col_right=columns_right[2])
    #Printing the results:
    print_steps_results(df=df_inclusivity_left_merge,
                        one_col_right=columns_right[1],
                        step_name="Seconde étape : merge avec inclusion après nettoyage des données")

    
    
    
    #First method with TFIDF:
    df_first_tfidf_merge=find_return_answers(df_missing_left=df_inclusivity_left_merge, 
                                             df_right=df_right_travail, 
                                             column_left=columns_left[1],
                                             column_right=columns_right[1], 
                                             other_column_left=columns_left[2], 
                                             other_column_right=columns_right[2], 
                                             top_n=5)
    

    #Printing the results:
    print_steps_results(df=df_first_tfidf_merge,
                        one_col_right=columns_right[1],
                        step_name="Première étape de TFIDF")
    
    


        
    #Second method with TFIDF:
    df_second_tfidf_merge=find_return_answers_deux(df_missing_left=df_first_tfidf_merge, 
                                                   df_right=df_right_travail, 
                                                   column_left=columns_left[1],
                                                   column_right=columns_right[1], 
                                                   other_column_left=columns_left[2], 
                                                   other_column_right=columns_right[2], 
                                                   top_n=5)
    #Printing the results:
    print_steps_results(df=df_second_tfidf_merge,
                        one_col_right=columns_right[1],
                        step_name="Seconde étape de TFIDF")
    
    
    return df_second_tfidf_merge

## Application et test de la fonction

### Import des Datasets

In [9]:
#Base des fréquentations:
df_freq=pd.read_csv('data_clean/df_freq_clean.csv')
#Base Muséofile:
df_museofile=pd.read_csv('data_clean/df_geo_museofile.csv')

In [10]:
df_freq.head()

,REF_DU_MUSEE,REGION,NOM_DU_MUSEE,VILLE,Type_de_fréquentation,Année,Entrées
0,7511701,ÎLE-DE-France,Musée National Jean-Jacques Henner,PARIS,Payante,2008,0.0
1,7511701,ÎLE-DE-France,Musée National Jean-Jacques Henner,PARIS,Payante,2002,803.0
2,7511701,ÎLE-DE-France,Musée National Jean-Jacques Henner,PARIS,Payante,2004,0.0
3,7511701,ÎLE-DE-France,Musée National Jean-Jacques Henner,PARIS,Payante,2010,8729.0
4,7511801,ÎLE-DE-France,Musée de Montmartre,PARIS,Payante,2009,50541.0


In [11]:
df_museofile.head(2)

,Identifiant,Adresse,Artiste,Atout,Catégorie,Code_Postal,Domaine_thématique,Département,Date_de_saisie,Histoire,...,Région,Téléphone,Thèmes,URL,Ville,geolocalisation,coord_x,coord_y,merc_x,merc_y
0,M0410,"Parc de Bécon, 178 Boulevard Saint Denis","Ary Scheffer, René Ménard, Fernand Roybet, Car...",Situé dans le parc de Bécon dont l’origine rem...,Maison d'artiste,92400,Arts décoratifs;Art moderne et contemporain;Be...,Hauts-de-Seine,2019-01-21,Les collections furent constituées au départ p...,...,Ile-de-France,01 71 05 77 92,"Beaux-Arts : Dessin, Estampe et Affiche, Peint...",http://www.museeroybetfould.fr,Courbevoie,"48.900998,2.271279",48.900998,2.271279,252837.621729,6.258079e+06
1,M5076,Place Monsenergue,"Joseph Vernet, Pascal de La Rose, Morel-Fatio,...",Conservatoire de l'histoire de l'arsenal et du...,NaN,83000,Histoire,Var,2019-01-21,Créé à la fin du Premier Empire et ouvert au p...,...,Provence-Alpes-Côte d'Azur,04 94 02 02 01,"Archéologie nationale : Gallo-romain, Moderne;...",http://www.musee-marine.fr,Toulon,"43.122018,5.929368",43.122018,5.929368,660054.226486,5.330563e+06


### Application

Nous n'avons plus qu'à spécifier les arguments de a fonction : 

In [12]:
df_left=df_museofile
df_right=df_freq
columns_left=['Identifiant', 'Nom_officiel', 'Ville']
columns_right=['REF_DU_MUSEE', 'NOM_DU_MUSEE', 'VILLE']

Puis la lancer:

In [13]:
df_results=merging_two_datasets(df_left, df_right, columns_left, columns_right)

------------------ Première étape : merge simple après nettoyage des données -------------------
Au début de cette étape, il y avait 1114 éléments à trouver.
    Appliquer cette étape a permis de trouver 664 éléments supplémentaires.

Il reste maintenant 450 éléments à trouver.
------------------ Seconde étape : merge avec inclusion après nettoyage des données -------------------
Au début de cette étape, il y avait 1114 éléments à trouver.
    Appliquer cette étape a permis de trouver 857 éléments supplémentaires.

Il reste maintenant 257 éléments à trouver.
------------------ Première étape de TFIDF -------------------
Au début de cette étape, il y avait 1114 éléments à trouver.
    Appliquer cette étape a permis de trouver 1025 éléments supplémentaires.

Il reste maintenant 89 éléments à trouver.
------------------ Seconde étape de TFIDF -------------------
Au début de cette étape, il y avait 1114 éléments à trouver.
    Appliquer cette étape a permis de trouver 1089 éléments supplém

Les résultats sont alors ici stockés dans `df_results`.

### Résultats

Nous avons pu voir que la fonction renvoie un affichage en temps réel (i.e. en même temps que l'execution de la fonction) des résultats. 

Les résultats sont les mêmes que lorsque j'ai développé les fonctions sans que ce soit automatique, ce qui est logique.

Vous pouvez d'ailleurs trouver ce notebook de développement [ici](https://github.com/adrihans/musees_de_france_app/blob/master/Linking_datasets.ipynb).

#### Temps nécessaire ici à l'éxécution de la fonction (sans interractions):

In [14]:
%time df_results=merging_two_datasets(df_left, df_right, columns_left, columns_right)

------------------ Première étape : merge simple après nettoyage des données -------------------
Au début de cette étape, il y avait 1114 éléments à trouver.
    Appliquer cette étape a permis de trouver 664 éléments supplémentaires.

Il reste maintenant 450 éléments à trouver.
------------------ Seconde étape : merge avec inclusion après nettoyage des données -------------------
Au début de cette étape, il y avait 1114 éléments à trouver.
    Appliquer cette étape a permis de trouver 857 éléments supplémentaires.

Il reste maintenant 257 éléments à trouver.
------------------ Première étape de TFIDF -------------------
Au début de cette étape, il y avait 1114 éléments à trouver.
    Appliquer cette étape a permis de trouver 1025 éléments supplémentaires.

Il reste maintenant 89 éléments à trouver.
------------------ Seconde étape de TFIDF -------------------
Au début de cette étape, il y avait 1114 éléments à trouver.
    Appliquer cette étape a permis de trouver 1089 éléments supplém

La fonction ainsi développée a permis, dans cet exemple fleuve concernant la base Muséofile, de traiter les **1114** lignes en **2 min 25 s**.

#### Essai de la fonction avec interractions:

## Etapes possibles après l'utilisation de la fonction:

Après l'utilisation de la fonction, on peut extraire un DataFrame avec seulement deux colonnes, c'est à dire les identifiants respectifs des deux Dataframes de départ, afin de pouvoir relier les deux datasets de base via ce dataframe *'de jonction'*.

In [15]:
id_col_left='Identifiant'
id_col_right='REF_DU_MUSEE'

In [16]:
df_jonction_freq_museofile=df_results[[id_col_left,id_col_right]].dropna(axis=0)

In [17]:
df_jonction_freq_museofile.head()

,Identifiant,REF_DU_MUSEE
0,M0001,6702101
1,M0002,6733901
2,M0003,6706101
3,M0004,6718002
4,M0005,6718001


On peut maintenant enregistrer ce DataFrame de jonction:

In [18]:
df_jonction_freq_museofile.to_csv('data_clean/df_jonction_freq_museofile.csv', index=False)

## Conclusion

L'utilisation de ce notebook, et plus précisement de la fonction globale définie et testée, permet très facilement de définir les relations entre deux datasets. 

En effet, la jonction peut alors s'établir en précisant seulement: 
- Les deux jeux de données, sous format Pandas DataFrame
- Les colonnes utiles 


Et on obtient un nouveau dataset permettant de faire la jonction entre les deux. 


Les réultats sont très satisfaisants, et même s'il y a des erreurs imputables à l'utilisation de méthodes plus évoluées comme TF-IDF, cela peut être résolu par l'utilisation des interactions permises par la fonction globale. 

De plus, les résultats s'établissent rapidement, avec un temps de 2min25s pour un DataFrame contenant 1114 lignes.

In [37]:
df_freq.isna().sum()

REF_DU_MUSEE              0
REGION                    0
NOM_DU_MUSEE              0
VILLE                     0
Type_de_fréquentation     0
Année                     0
Entrées                  97
dtype: int64

In [38]:
df_freq.fillna(0,inplace=True)

In [45]:
df_freq['Entrées']=df_freq['Entrées'].astype(int)

In [46]:
df_freq.to_csv('data_clean/df_freq_clean_without_nan.csv',index=False)

In [44]:
df_freq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59876 entries, 0 to 59875
Data columns (total 7 columns):
REF_DU_MUSEE             59876 non-null object
REGION                   59876 non-null object
NOM_DU_MUSEE             59876 non-null object
VILLE                    59876 non-null object
Type_de_fréquentation    59876 non-null object
Année                    59876 non-null int64
Entrées                  59876 non-null float64
dtypes: float64(1), int64(1), object(5)
memory usage: 3.2+ MB
